# 快速入门 GPT-4 Vison

从历史上看，语言模型系统仅接受**文本**作为输入。但是单一的输入形式，限制了大模型的应用落地范围。

随着技术发展，OpenAI 开发的 GPT-4 Turbo with Vision（简称 GPT-4V）允许模型接收**图像**作为输入，并回答关于它们的问题。

📢注意，目前在 Assistants API 中使用 GPT-4 时还不支持图像输入。

## 使用 GPT-4V 识别线上图像（URL）

![image_sample](https://upload.wikimedia.org/wikipedia/commons/thumb/d/dd/Gfp-wisconsin-madison-the-nature-boardwalk.jpg/2560px-Gfp-wisconsin-madison-the-nature-boardwalk.jpg)

In [16]:
from openai import OpenAI

client = OpenAI()

response = client.chat.completions.create(
  model="gpt-4-turbo",
  messages=[
    {
      "role": "user",
      "content": [
        {"type": "text", "text": "介绍下这幅图?"},
        {
          "type": "image_url",
          "image_url": {
            "url": "https://upload.wikimedia.org/wikipedia/commons/thumb/d/dd/Gfp-wisconsin-madison-the-nature-boardwalk.jpg/2560px-Gfp-wisconsin-madison-the-nature-boardwalk.jpg",
          },
        },
      ],
    }
  ],
  max_tokens=300,
)

print(response.choices[0])

Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='这幅图展示了一条木制的小路穿过一片广阔的草地，通向远处的树林。天空蓝天白云，景色开阔明亮，草地绿意盎然，给人一种宁静祥和的感觉。整幅图呈现出一种自然美景，似乎是一个适合散步和享受大自然的地方。', role='assistant', function_call=None, tool_calls=None))


In [17]:
response.choices[0].message.content

'这幅图展示了一条木制的小路穿过一片广阔的草地，通向远处的树林。天空蓝天白云，景色开阔明亮，草地绿意盎然，给人一种宁静祥和的感觉。整幅图呈现出一种自然美景，似乎是一个适合散步和享受大自然的地方。'

### 封装成一个函数 query_image_description

In [18]:
def query_image_description(url, prompt="介绍下这幅图?"):
    client = OpenAI()  # 初始化 OpenAI 客户端
    
    # 发送请求给 OpenAI 的聊天模型
    response = client.chat.completions.create(
        model="gpt-4-turbo",  # 指定使用的模型
        messages=[
            {
                "role": "user",
                "content": [
                    {"type": "text", "text": prompt},
                    {"type": "image_url", "image_url": {"url": url}},
                ],
            }
        ],
        max_tokens=300,
    )
    
    # 返回模型的响应
    return response.choices[0].message.content


In [19]:
### 调用函数测试

![meme_0](https://p6.itc.cn/q_70/images03/20200602/0c267a0d3d814c9783659eb956969ba1.jpeg)

'[meme_0]' is not recognized as an internal or external command,
operable program or batch file.


In [20]:
image_url = "https://p6.itc.cn/q_70/images03/20200602/0c267a0d3d814c9783659eb956969ba1.jpeg"
content = query_image_description(image_url)
print(content)

这幅图是一张幽默的搞笑图片，通过夸张的对比展示了两只狗的不同健身成果。图中左侧的狗被描绘为具有非常发达的肌肉，类似于健身运动员的身材，上面的文字“16岁的我，工作后的我”表明这可能是一个人在不同阶段的幻想与现实的对比。左侧狗的四个标签描述了其强壮的特点：“我刚健身一岁半哦”、“全球防暑”、“身体棒极了”和“人见人爱的大胖”。

右侧的狗则显得较为普通，肌肉没有那么发达，坐着的姿势看起来有些无奈和沮丧。其四个标签则描述了其平凡或略显怯懦的特性：“好累别摸我”、“好想吃增肌粉”、“我天生体质不行”和“


### 使用 GPT-4V 识别本地图像文件（Base64编码）


In [30]:
from openai import OpenAI
import base64
import requests
import json

client = OpenAI(
    base_url="https://api.xiaoai.plus/v1",
    api_key="sk-dzZ2slQ3Qvb64c4d8cFdA752Dc6e4e4086973f5bEc2195C9"
)  # 初始化 OpenAI 客户端

def query_base64_image_description(image_path, prompt="解释下图里的内容？", max_tokens=1000):

    # 实现 Base64 编码
    def encode_image(path):
        with open(path, "rb") as image_file:
            return base64.b64encode(image_file.read()).decode('utf-8')

    # 获取图像的 Base64 编码字符串
    base64_image = encode_image(image_path)

    # 构造请求的 HTTP Header
    headers = {
        "Content-Type": "application/json",
        "Authorization": f"Bearer {client.api_key}"
    }

    # 构造请求的负载
    payload = {
        "model": "gpt-4-turbo",
        "messages": [
            {
                "role": "user",
                "content": [
                    {"type": "text", "text": prompt},
                    {"type": "image_url", "image_url": {"url": f"data:image/jpeg;base64,{base64_image}"}}
                ]
            }
        ],
        "max_tokens": max_tokens
    }

    # 发送 HTTP 请求
    response = requests.post("https://api.openai.com/v1/chat/completions", headers=headers, json=payload)

    # 检查响应并提取所需的 content 字段
    if response.status_code == 200:
        response_data = response.json()
        content = response_data['choices'][0]['message']['content']
        return content
    else:
        return f"Error: {response.status_code}, {response.text}"

#### 使用 Assistants API生成的 GDP 40年对比曲线图

![gdp_data](./images/gdp_1980_2020.jpg)

content = query_base64_image_description("./images/gdp_1980_2020.jpg")
print(content)

#### 使用 GPT-4V 识别手写体笔记

![](./images/handwriting_0.jpg)

In [27]:
content = query_base64_image_description("./images/handwriting_0.jpg")
print(content)

ConnectTimeout: HTTPSConnectionPool(host='api.openai.com', port=443): Max retries exceeded with url: /v1/chat/completions (Caused by ConnectTimeoutError(<urllib3.connection.HTTPSConnection object at 0x0000022521168580>, 'Connection to api.openai.com timed out. (connect timeout=None)'))

#### 在 Jupyter 标准输出中渲染 Markdown 格式内容

In [ ]:
from IPython.display import display, Markdown

# 使用 display 和 Markdown 函数显示 Markdown 内容
display(Markdown(content))

![](./images/handwriting_1.jpg)

In [ ]:
content = query_base64_image_description("./images/handwriting_1.jpg")
display(Markdown(content))

## Homework: 


### #1

使用 GPT-4V 识别带有手写体文字的本地图像文件，分享结果。

### #2

整合 `query_base64_image_description` 函数和 Markdown 格式渲染方法，使得输出结果更易阅读。

In [ ]:
content_test = query_base64_image_description("./images/finn_test.jpg")
print(content_test)

这张图片展示了一张写有一首诗的纸。上面写着著名诗人杜牧的“清明”，下面是诗中句子：

-
清明时节雨纷纷
路上行人欲断魂
借问酒家何处有
牧童遥指杏花村

这张纸条可能是某人在对杜牧的诗进行书写与记忆。

In [ ]:
from IPython.display import display, Markdown

# 使用 display 和 Markdown 函数显示 Markdown 内容
display(Markdown(content_test))